In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

In [ ]:
torch.cuda.empty_cache()

In [ ]:
transfrom_model = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomAffine(0, translate = (0.1, 0.1)),
    transforms.ToTensor(),
])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root = '.', train = True, transform = transfrom_model, download = True)
test_dataset = torchvision.datasets.CIFAR10(root = '.', train = False, transform = transfrom_model, download = True)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 128, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = 128, shuffle = False)

In [ ]:
train_dataset.data.shape

In [ ]:
train_dataset.data.max()

In [ ]:
num_classes = len(set(train_dataset.targets))
num_classes

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2)
        )

        self.flatten = nn.Flatten(1)

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(128 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = F.dropout(x, p = 0.2)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p = 0.2)
        out = self.fc2(x)
        return out

In [ ]:
model = CNN(num_classes = num_classes)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [ ]:
from torchsummary import summary
summary_model = summary(model, (3, 32, 32))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
total_len = (len(train_loader)) * 2 + len(test_loader)
total_len

In [ ]:
n_epochs = 15
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)
train_accuracies = np.zeros(n_epochs)
test_accuracies = np.zeros(n_epochs)

for epoch in range(n_epochs):
    epoch_str = str(epoch + 1).rjust(len(str(n_epochs)), " ")
    with tqdm(total = total_len, desc = f"Epoch [ {epoch_str}/{n_epochs} ] : ") as pbar:
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()
            pbar.update(1)

        n_correct = 0
        n_total = 0
        train_loss = []
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs_train = model(inputs)
            _, predictions_train = torch.max(outputs_train, 1)
            n_correct += (predictions_train == targets).sum().item()
            n_total += targets.shape[0]
            loss_train = criterion(outputs_train, targets)
            train_loss.append(loss_train.item())
            pbar.update(1)

        train_acc_epoch = n_correct / n_total
        train_loss = np.mean(train_loss)
        train_losses[epoch] = train_loss
        train_accuracies[epoch] = train_acc_epoch

        n_correct = 0
        n_total = 0
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs_test = model(inputs)
            _, predictions_test = torch.max(outputs_test, 1)
            n_correct += (predictions_test == targets).sum().item()
            n_total += targets.shape[0]
            loss_test = criterion(outputs_test, targets)
            test_loss.append(loss_test.item())
            pbar.update(1)

        test_acc_epoch = n_correct / n_total
        test_loss = np.mean(test_loss)
        test_losses[epoch] = test_loss
        test_accuracies[epoch] = test_acc_epoch
        pbar.set_description(f"Epoch [ {epoch_str}/{n_epochs} ] ")
        pbar.set_postfix({'Train Accuracy' : f"{train_acc_epoch:.4f}", 'Train Loss' : f"{train_loss:.4f}", 'Test Accuracy' : f"{test_acc_epoch:.4f}", 'Test Loss' : f"{test_loss:.4f}"})

In [ ]:
plt.plot(train_accuracies, label = 'Train Accuracy')
plt.plot(test_accuracies, label = 'Test Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_losses, label = 'Train Loss')
plt.plot(test_losses, label = 'Test Loss')
plt.legend()
plt.show()

In [ ]:
n_correct = 0
n_total = 0
with tqdm(total = (len(train_loader) + len(test_loader)), desc = "Calculating accuracies : ") as pbar:
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs_train_final = model(inputs)
        _, predictions_train_final = torch.max(outputs_train_final, 1)
        n_correct += (predictions_train_final == targets).sum().item()
        n_total += targets.shape[0]
        pbar.update(1)

    train_acc = n_correct / n_total

    n_correct = 0
    n_total = 0
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs_test_final = model(inputs)
        _, predictions_test_final = torch.max(outputs_test_final, 1)
        n_correct += (predictions_test_final == targets).sum().item()
        n_total += targets.shape[0]
        pbar.update(1)

    test_acc = n_correct / n_total
    pbar.set_description("Process completed ")
    # pbar.set_postfix_str(f"Train Accuracy : {train_acc:.4f}, Test Accuracy : {test_acc:.4f}")  
    print(f"\nTrain Accuracy : {train_acc:.4f}, Test Accuracy : {test_acc:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

In [ ]:
x_test = test_dataset.data
y_test = np.array(test_dataset.targets)
p_test = np.array([])

for inputs, targets in test_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    outputs_cm = model(inputs)
    _, prediction_cm = torch.max(outputs_cm, 1)
    p_test = np.concatenate((p_test, prediction_cm.cpu().numpy()))

cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [ ]:
labels = test_dataset.classes

In [ ]:
p_test = p_test.astype(np.uint8)
misclassified_idx = np.where(p_test != y_test)[0]
sample_idx = np.random.choice(misclassified_idx, 10, replace=False)
plt.figure(figsize=(32,32))
for j, i in enumerate(sample_idx):
  plt.subplot(10, 1, j + 1)
  plt.axis('off')
  plt.imshow(x_test[i].reshape(32,32,3))
  plt.title("True label: %s Predicted: %s" % (labels[y_test[i]], labels[p_test[i]]));